In [1]:
#importing Necessary Libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('medical.csv')

In [3]:
df.head()

age     sex   bmi  children smoker     region   charges
0   56    male  37.2         2     no  northeast  28352.87
1   46  female  20.9         4     no  southeast  21278.14
2   32    male  26.6         1     no  southeast  18100.51
3   60  female  32.3         2     no  southeast  29908.80
4   25    male  39.4         3    yes  northeast  42870.13

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1000 non-null   int64  
 1   sex       1000 non-null   object 
 2   bmi       1000 non-null   float64
 3   children  1000 non-null   int64  
 4   smoker    1000 non-null   object 
 5   region    1000 non-null   object 
 6   charges   1000 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 54.8+ KB


In [5]:
df.describe()

age          bmi    children      charges
count  1000.000000  1000.000000  1000.00000   1000.00000
mean     41.575000    30.340100     2.43900  26300.63057
std      13.765677     5.973606     1.70798   8843.71197
min      18.000000    12.500000     0.00000  10717.98000
25%      30.000000    26.300000     1.00000  20289.32000
50%      42.000000    30.250000     2.00000  24249.65500
75%      53.000000    34.300000     4.00000  28637.36250
max      65.000000    49.200000     5.00000  54401.66000

In [6]:
df.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [7]:
# Add Gaussian noise to simulate real-world variance
np.random.seed(42)
df['charges'] += np.random.normal(0, 3500, size=len(df))  # increase or decrease 5000 as needed


In [8]:
# One Hot Encoding
dataset = pd.get_dummies(data=df, drop_first=True)

In [9]:
x= dataset.drop(columns='charges')
y =dataset['charges']

In [10]:
x

age   bmi  children  sex_male  smoker_yes  region_northwest  \
0     56  37.2         2      True       False             False   
1     46  20.9         4     False       False             False   
2     32  26.6         1      True       False             False   
3     60  32.3         2     False       False             False   
4     25  39.4         3      True        True             False   
..   ...   ...       ...       ...         ...               ...   
995   23  27.2         0     False       False             False   
996   43  18.1         2     False       False             False   
997   21  34.5         0      True       False             False   
998   36  23.6         0     False        True             False   
999   37  31.4         1     False        True             False   

     region_southeast  region_southwest  
0               False             False  
1                True             False  
2                True             False  
3                True             False  
4               False             False  
..                ...               ...  
995             False              True  
996              True             False  
997             False              True  
998             False             False  
999             False             False  

[1000 rows x 8 columns]

In [11]:
y

0      30091.369536
1      20794.214946
2      20367.419883
3      35239.404497
4      42050.593188
           ...     
995    14654.358975
996    23986.682844
997    19203.020014
998    34720.553536
999    42791.219735
Name: charges, Length: 1000, dtype: float64

In [12]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2,random_state=0)

In [13]:
regressor_xgb = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=4, random_state=0)
regressor_xgb.fit(x_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.1, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=4,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=100,
             n_jobs=None, num_parallel_tree=None, ...)

In [22]:
from sklearn.metrics import r2_score,mean_squared_error, mean_absolute_error

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("R² Score:", round(r2, 4))
print("MAE:", round(mae, 2))
print("RMSE:", round(rmse, 2))

R² Score: 0.8377
MAE: 3259.49
RMSE: 4095.31


In [23]:
import joblib
joblib.dump(regressor_xgb, 'train.pkl')

['train.pkl']

In [20]:
obs = [[19, 27, 0, 0, 0, 1, 0, 0]]
print("Test prediction:", regressor_xgb.predict(obs))

Test prediction: [12591.21]
